In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("..")))
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from src.data_loader import load_sales, load_purchases, load_products, load_packaging
from src.load_from_db import load_sales_db, load_purchases_db, load_products_db, load_packaging_db
from src.demand_forecast import calculate_forecast
from src.purchase_suggester import generate_purchase_suggestion,export_to_excel


In [ ]:
# === Opciones disponibles ===
coverage_days_options = [7, 15, 21, 30]
forecast_methods = ["mean", "rolling", "weighted", "trend"]

# === Widgets ===
coverage_dropdown = widgets.Dropdown(
    options=coverage_days_options,
    value=7,
    description="Cobertura (días):",
    style={'description_width': 'initial'}
)

method_dropdown = widgets.Dropdown(
    options=forecast_methods,
    value="mean",
    description="Método forecast:",
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description="Generar Sugerido",
    button_style="success"
)

output = widgets.Output()

# === Función a ejecutar ===
def run_forecast(b):
    with output:
        output.clear_output()
        try:
            coverage_days = coverage_dropdown.value
            method = method_dropdown.value

            print(f"⚡ Ejecutando forecast con {coverage_days} días y método '{method}'...\n")

            # === Cargar datos ===
            '''sales = load_sales("../data/sales.csv")
            products = load_products("../data/products.csv")      # incluye stock y datos producto
            purchases = load_purchases("../data/purchases.csv")   # historial de compras
            packaging = load_packaging("../data/packaging.csv")'''

            sales = load_sales_db()
            products = load_products_db()
            purchases = load_purchases_db()
            packaging = load_packaging_db()


            # === Forecast (ventas diarias) ===
            forecast = calculate_forecast(
                sales,
                days=60,   # usa últimos 60 días para suavizar outliers
                method=method,
                window=7
            )

            # === Generar sugerido ===
            suggestion = generate_purchase_suggestion(
                sales_df=sales,
                forecast_df=forecast,
                stock_df=products,      # productos contiene current_stock
                products_df=products,   # productos también tiene code y name
                purchases_df=purchases,
                packaging_df= packaging,
                coverage_days=coverage_days,
                safety_days=2           # stock mínimo para 2–3 días de seguridad
            )


            if suggestion.empty:
                print("⚠️ No hay datos para mostrar (revisa tus CSV o filtros).")
            else:
                display(suggestion.head(15))

                output_path = f"../output/purchase_suggestion_{pd.Timestamp.now().strftime('%Y%m%d_%H%M')}.xlsx"
                export_to_excel(suggestion, output_path)
                print(f"\n✅ Reporte generado exitosamente en:\n{output_path}")

        except Exception as e:
            print("❌ Error en la ejecución:", e)

# Conectar botón a función
run_button.on_click(run_forecast)



# Mostrar widgets
display(coverage_dropdown, method_dropdown, run_button, output)